# <center>Assignment 2</center>

This assignment is based on the tensorflow tutorial. You can choose to code in Python2 or Python3. All the imports made in this notebook are as below; if these imports work, you are (mostly) set to complete the assignment.

In [2]:
from __future__ import print_function, division
import random 
import tensorflow as tf
import numpy as np

## Tensorflow - Matrix Operations

All the following questions should be solved without using any of python's in-built control structures : for, while, if-else, etc. In addition to the tutorial material, you may find these functions useful:
* tf.cast
* tf.range
* tf.pow

Also, the following functions are used in later part of the assignment: tf.sqrt and tf.square. 

### 1. Alternating addition

In [3]:
def alt_addition():
    """
        Given a 1d-matrix of 1 x n, compute the alternating addition.
        Ex: If A is
        1 2 3 4 5 6 7 8 9
        the return value should be 1 - 2 + 3 - 4 + 5 - 6 + 7 - 8 + 9.
        Your solution shouldn't include the use of tf.where or tf.boolean_mask. 
        It should include only mathematical operations.
    """
    A = tf.placeholder(dtype=tf.float64, shape=[1, None])
    B = tf.subtract(tf.reduce_sum(A[0][::2]), tf.reduce_sum(A[0][1::2]))
    return A, B

In [4]:
A, B = alt_addition()

In [5]:
# Just run this cell after completing the function alt_addition.
def compute_alt(array):
    sum_ = 0
    for index, i in enumerate(array):
        if index % 2 == 0:
            sum_ += i
        else:
            sum_ -= i
    return sum_
sess = tf.InteractiveSession()
for num in [5, 10, 13, 20, 100]:
    x = np.random.randn(num)
    b = sess.run(B, feed_dict={A: x.reshape([1, -1])})
    sum_ = compute_alt(x)
    if abs(b - sum_) < 10**-10:
        print("Correct!")
    else:
        print("Incorrect.")

Correct!
Correct!
Correct!
Correct!
Correct!


## 2. Odd Square and Even Cube

In [6]:
def sq_cube():
    """
    Given a 2-d matrix of dimensions N x N, square the odd elements and cube the even elements.
    Your solution shouldn't include the use of tf.where or tf.boolean_mask. 
    It should include only mathematical operations.
    """
    A = tf.placeholder(dtype=tf.float64, shape=[None, None])
    
    # flag all odd and even elements in A
    odds = A % 2
    evens = tf.abs(A % 2 - 1)
    
    # fill in values from A
    odds = odds * A
    evens = evens * A
    
    # compute the squared and cubed values accordingly
    odds = tf.square(odds)
    evens = tf.pow(evens, 3)
    
    # merge them back together
    B = odds + evens
    
    return A, B

In [7]:
A, B = sq_cube()

In [9]:
# Just run this cell after completing the function sq_cube.
def compute_sq_cube(array):
    m = [[0 for _ in range(len(array[0]))] for _ in range(len(array))]
    for row_ in range(len(array)):
        for col_ in range(len(array[row_])):
            m[row_][col_] = array[row_][col_]**3 if array[row_][col_] % 2 == 0 else array[row_][col_]**2
    return m
sess = tf.InteractiveSession()
for num in [5, 10, 20]:
    x = np.array(range(num * num)).reshape([num, num])
    b = sess.run(B, feed_dict={A: x})
    res = compute_sq_cube(x.tolist())
    if (abs(b - res) < 10**-10).all():
        print("Correct!")
    else:
        print("Incorrect.")

Correct!
Correct!
Correct!


## 3. Tensorflow differentiation

In this question, you will learn about tf.gradients. Essentially, tf.gradients(p,q) gives dp/dq. 

In [55]:
x = tf.placeholder(tf.float32)
y = tf.sqrt(tf.square(x))
dy_dx = tf.gradients(y, x)

Essentially, y = x. And, as you know dy/dx = 1.  

In [58]:
sess = tf.Session()
sess.run(dy_dx, feed_dict={x: 10})

[1.0]

And it is 1! Now, let's try for some other values

In [56]:
sess.run(dy_dx, feed_dict={x: 0})

[nan]

In [50]:
sess.run(dy_dx, feed_dict={x: 10**-20})

[inf]

Oops! We got nan and inf even though the derivative(limit) exists!! Explain why.

<b>Answer 1 x=0:</b> <i>Let f(x)=sqrt(x^2^). Then f'(x)=x/sqrt(x^2^) by the chain rule. When x=0, Tensorflow's computational graph notices that the denominator is 0, and hence it produces a NaN result because we can't divide by 0. Tensorflow does not simplify f'(x), which is why we get a NaN result here. It is also important to notice that f'(x) does not exist at x=0 from a mathematical standpoint.</i>

<b>Answer 2 x=10^-20^:</b> <i>The placeholder x has the data type float32, which is a 32-bit single-precision floating point. The minimum positive normal value that can be stored in a float32 data type is 2^-126^ which is approximately equal to 1.18 * 10^-38^. When computing sqrt(x^2^) to get y, we are first computing x^2^, and x^2^ for x=10^-20^ is equal to 10^-40^, which is smaller than the minimum value of float32. This means that it overflowed / went of of bounds of the single precision floating point, and the exponent will be all 1's and and the fraction will be all 0's, which by the definition of IEEE single precision floating points results in infinity.</i>


Further, come up with another function of x which shows similar behavior and demonstrate it below. Do not use trivial modifications of the above function like (x\*\*3)**1/3. In other words, demonstrate a function which gives the derivative as equal to nan/inf even though the derivative is defined. 

In [59]:
x = tf.placeholder(tf.float32)
y = tf.divide(tf.sin(x), x)
dy_dx = tf.gradients(y, x)

In [60]:
sess = tf.Session()
sess.run(dy_dx, feed_dict={x: 10})

[-0.078466937]

In [61]:
sess.run(dy_dx, feed_dict={x: 0})

[nan]

## Links

- http://organicchemist.us/organichem/my_pages/absvalder.html
- Section 3.3 at: http://www.toves.org/books/float/
- https://en.wikipedia.org/wiki/Single-precision_floating-point_format
- https://www.tensorflow.org/api_docs/python/tf/DType
- http://www.wolframalpha.com/input/?i=derivative+sqrt(x%5E2)
